In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import time

from dgl.geometry import farthest_point_sampler
import open3d as o3d

from utils import load_yaml, fps, fps_rad, recenter, depth2fgpcd, pcd2pix, fps_np, downsample_pcd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
raw_obs = np.load('raw_obs.npy')
raw_obs.shape

(21, 720, 720, 5)

In [3]:
camera_params = np.load('cam_params.npy')
camera_params

array([869.11683083, 869.11683083, 360.        , 360.        ])

In [ ]:
# def obs2ptcl_fixed_num_batch(obs, particle_num, batch_size):
#         # print('obs', obs.shape) # carrots: (720, 720, 5)
#         global_scale = 24
#         depth = obs[..., -1] / global_scale
#         # print('depth', depth.shape) # (720, 720)
        
#         batch_sampled_ptcl = np.zeros((batch_size, particle_num, 3))
#         batch_particle_r = np.zeros((batch_size, ))
#         for i in range(batch_size):
#             fgpcd = depth2fgpcd(depth, depth<0.599/0.8, get_cam_params())
#             fgpcd = downsample_pcd(fgpcd, 0.01)
#             sampled_ptcl, particle_r = fps(fgpcd, particle_num)
#             batch_sampled_ptcl[i] = recenter(fgpcd, sampled_ptcl, r = min(0.02, 0.5 * particle_r))
#             batch_particle_r[i] = particle_r
#         return batch_sampled_ptcl, batch_particle_r

In [7]:
global_scale = 24
obs = raw_obs[10]
depth = obs[..., -1] / global_scale
fgpcd = depth2fgpcd(depth, depth<0.599/0.8, camera_params)
# fgpcd = downsample_pcd(fgpcd, 0.01)

In [8]:
print(depth.shape)
print(fgpcd.shape)

(720, 720)
(52501, 3)


In [5]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(fgpcd)
o3d.visualization.draw_geometries([pcd])

In [17]:
# point cloud video
vis = o3d.visualization.Visualizer()
vis.create_window()

# geometry is the point cloud used in your animaiton
geometry = o3d.geometry.PointCloud()
vis.add_geometry(geometry)

global_scale = 24
particle_num = 50 # obtained from resolution regressor 
save_images = True

for i in range(len(raw_obs)):
    obs = raw_obs[i]
    depth = obs[..., -1] / global_scale
    
    fgpcd = depth2fgpcd(depth, depth<0.599/0.8, camera_params)
    fgpcd = downsample_pcd(fgpcd, 0.01)
    sampled_ptcl, particle_r = fps(fgpcd, particle_num) 
    fgpcd = sampled_ptcl
    
    geometry.points = o3d.utility.Vector3dVector(fgpcd)
    vis.add_geometry(geometry)
    vis.poll_events()
    vis.update_renderer()
    
    if save_images:
        vis.capture_screen_image("imgs/{:04d}.jpg".format(i))
    
    time.sleep(0.3)

vis.destroy_window()

In [16]:
# imgs to video
import glob 
import os

img_array = []
for filename in sorted(glob.glob(f'./imgs/*.jpg'), key=os.path.getmtime):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

out = cv2.VideoWriter(f'ptcl_sampled.mp4',cv2.VideoWriter_fourcc(*'mp4v'), 5, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()
